# Formatos de Archivo para Big Data

## Objetivos de Aprendizaje
- Conocer los principales formatos de archivo
- Comparar rendimiento entre formatos
- Elegir el formato adecuado para cada caso
- Configurar compresión y opciones de escritura

## Prerequisitos
- `00_setup/02_spark_basics.ipynb`

## Tiempo Estimado
⏱️ 45 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 8: Data Storage - Formatos optimizados para S3

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import time
import os

spark = SparkSession.builder \
    .appName("FileFormats") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("Spark listo")

In [ ]:
# Generar datos de prueba
# 100,000 registros para comparar formatos

datos = [(i, 
          f"producto_{i % 1000}", 
          f"categoria_{i % 50}",
          float(i * 1.5),
          i % 100,
          f"descripcion larga del producto numero {i} con texto adicional"
         ) for i in range(100000)]

df = spark.createDataFrame(
    datos,
    ["id", "producto", "categoria", "precio", "stock", "descripcion"]
)

print(f"Registros: {df.count():,}")
df.printSchema()

---
# === SECCIÓN 1 ===
## 1. Formatos de Texto (CSV, JSON)

### Explicación Conceptual

**CSV (Comma-Separated Values)**
- ✅ Simple, legible, universal
- ❌ Sin esquema, sin compresión nativa, lento

**JSON (JavaScript Object Notation)**
- ✅ Esquema flexible, anidado, legible
- ❌ Verboso, grande, parsing lento

In [ ]:
# Rutas de salida
base_path = "/home/jovyan/data/sample/formatos"
os.makedirs(base_path, exist_ok=True)

# Escribir CSV
start = time.time()
df.write.mode("overwrite").option("header", "true").csv(f"{base_path}/csv")
tiempo_csv_write = time.time() - start

# Escribir JSON
start = time.time()
df.write.mode("overwrite").json(f"{base_path}/json")
tiempo_json_write = time.time() - start

print(f"CSV escritura: {tiempo_csv_write:.2f}s")
print(f"JSON escritura: {tiempo_json_write:.2f}s")

In [ ]:
# Leer CSV
start = time.time()
df_csv = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{base_path}/csv")
_ = df_csv.count()  # Forzar lectura
tiempo_csv_read = time.time() - start

# Leer JSON
start = time.time()
df_json = spark.read.json(f"{base_path}/json")
_ = df_json.count()
tiempo_json_read = time.time() - start

print(f"CSV lectura: {tiempo_csv_read:.2f}s")
print(f"JSON lectura: {tiempo_json_read:.2f}s")

---
# === SECCIÓN 2 ===
## 2. Formatos Columnares (Parquet, ORC)

### Explicación Conceptual

**Parquet**
- ✅ Columnar, comprimido, rápido para analytics
- ✅ Esquema embebido, predicate pushdown
- El estándar para data lakes

**ORC (Optimized Row Columnar)**
- ✅ Similar a Parquet, optimizado para Hive
- ✅ Mejor compresión en algunos casos

**¿Por qué columnar es mejor para analytics?**
- Solo lee columnas necesarias
- Mejor compresión (datos similares juntos)
- Estadísticas por columna para filtrar

In [ ]:
# Escribir Parquet
start = time.time()
df.write.mode("overwrite").parquet(f"{base_path}/parquet")
tiempo_parquet_write = time.time() - start

# Escribir ORC
start = time.time()
df.write.mode("overwrite").orc(f"{base_path}/orc")
tiempo_orc_write = time.time() - start

print(f"Parquet escritura: {tiempo_parquet_write:.2f}s")
print(f"ORC escritura: {tiempo_orc_write:.2f}s")

In [ ]:
# Leer Parquet
start = time.time()
df_parquet = spark.read.parquet(f"{base_path}/parquet")
_ = df_parquet.count()
tiempo_parquet_read = time.time() - start

# Leer ORC
start = time.time()
df_orc = spark.read.orc(f"{base_path}/orc")
_ = df_orc.count()
tiempo_orc_read = time.time() - start

print(f"Parquet lectura: {tiempo_parquet_read:.2f}s")
print(f"ORC lectura: {tiempo_orc_read:.2f}s")

In [ ]:
# Ventaja de columnar: leer solo columnas necesarias
print("Lectura selectiva de columnas:")

# CSV tiene que leer todo el archivo
start = time.time()
df_csv.select("id", "precio").filter(F.col("precio") > 1000).count()
tiempo_csv_select = time.time() - start

# Parquet solo lee columnas id y precio
start = time.time()
df_parquet.select("id", "precio").filter(F.col("precio") > 1000).count()
tiempo_parquet_select = time.time() - start

print(f"CSV (lee todo): {tiempo_csv_select:.3f}s")
print(f"Parquet (lee 2 columnas): {tiempo_parquet_select:.3f}s")

---
# === SECCIÓN 3 ===
## 3. Comparación de Tamaños

### Explicación Conceptual
El tamaño del archivo impacta directamente en:
- Costos de almacenamiento (S3)
- Tiempo de transferencia
- Velocidad de lectura

In [ ]:
import subprocess

def get_dir_size(path):
    """Obtener tamaño de directorio en bytes"""
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total += os.path.getsize(fp)
    return total

# Calcular tamaños
tamanos = {
    "CSV": get_dir_size(f"{base_path}/csv"),
    "JSON": get_dir_size(f"{base_path}/json"),
    "Parquet": get_dir_size(f"{base_path}/parquet"),
    "ORC": get_dir_size(f"{base_path}/orc")
}

print("Comparación de tamaños:")
print("-" * 40)
for formato, tamano in sorted(tamanos.items(), key=lambda x: x[1]):
    mb = tamano / (1024 * 1024)
    ratio = tamano / tamanos["CSV"]
    print(f"{formato:10} {mb:8.2f} MB  ({ratio:.1%} vs CSV)")

---
# === SECCIÓN 4 ===
## 4. Resumen Comparativo

In [ ]:
# Crear tabla resumen
resumen = spark.createDataFrame([
    ("CSV", tiempo_csv_write, tiempo_csv_read, tamanos["CSV"]/(1024*1024)),
    ("JSON", tiempo_json_write, tiempo_json_read, tamanos["JSON"]/(1024*1024)),
    ("Parquet", tiempo_parquet_write, tiempo_parquet_read, tamanos["Parquet"]/(1024*1024)),
    ("ORC", tiempo_orc_write, tiempo_orc_read, tamanos["ORC"]/(1024*1024))
], ["formato", "escritura_seg", "lectura_seg", "tamano_mb"])

print("RESUMEN COMPARATIVO:")
resumen.show()

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio F.1: Elegir Formato

¿Qué formato elegirías para cada caso?
1. Exportar datos para Excel
2. Data lake para analytics con Athena
3. Intercambio con una API REST
4. Streaming de eventos

In [ ]:
# Respuestas
print("1. Excel: CSV - Universal, fácil de abrir")
print("2. Athena: Parquet - Columnar, optimizado para queries")
print("3. API REST: JSON - Estándar para APIs")
print("4. Streaming: Avro - Esquema evolución, compacto")

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **CSV/JSON**: Texto, legible, lento, grande
- **Parquet/ORC**: Columnar, comprimido, rápido para analytics
- **Parquet** es el estándar para data lakes
- Formato columnar = solo lee columnas necesarias

### Conexión con AWS
- **S3**: Almacena todos los formatos
- **Athena**: Optimizado para Parquet/ORC en S3
- **Glue**: Puede convertir entre formatos
- **Redshift Spectrum**: Lee Parquet directamente

### Siguiente Paso
Continúa con: `02_partitioning.ipynb` para estrategias de particionamiento

In [ ]:
# Limpieza
import shutil
if os.path.exists(base_path):
    shutil.rmtree(base_path)
print("Archivos de prueba eliminados")